In [14]:
import numpy as np
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors.kde import KernelDensity
from scipy import stats
import time
from datetime import timedelta

In [2]:
dx = pd.read_excel('sameh_dx_all.xlsx')

In [3]:
dx

,SAID,Encounter_Hospital_Admission_Dat,_EncounterDischargeDate,Encounter_Type,Diagnosis_Start_date,Diagnosis_Event_Type,Diagnosis_End_Date,Diagnosis_Type,Primary_Coded_Diagnosis,Principal_Problem_Diagnosis,Diagnosis_Name,ICD9_Code,ICD9_Name,ICD10_Code,ICD10_Name,Diagnosis_Present_On_Admission
0,SA90200020380074,NaT,NaT,Office Visit,2012-04-18,EpicEncounterCsnId|DiagnosisId:EncounterDiagnosis,NaT,Encounter Diagnosis,No,No,Therapeutic drug monitoring,V58.83,Encounter for therapeutic drug monitoring,Z51.81,Encounter for therapeutic drug level monitoring,*Not Applicable
1,SA90200020380074,2012-04-18,2012-05-10,Hospital Encounter,2012-04-18,EpicHospitalAccountId|DiagnosisId:FinalDiagnosis,2012-05-10,Billing Diagnosis,No,No,Type I (juvenile type) diabetes mellitus witho...,250.01,Diabetes mellitus without mention of complicat...,E10.9,Type 1 diabetes mellitus without complications,Yes
2,SA90200020380074,2012-04-18,2012-05-10,Hospital Encounter,2012-04-18,EpicHospitalAccountId|DiagnosisId:FinalDiagnosis,2012-05-10,Billing Diagnosis,No,No,Unspecified systolic heart failure,428.2,Unspecified systolic heart failure,I50.20,Unspecified systolic (congestive) heart failure,Yes
3,SA90200020380074,2012-04-18,2012-05-10,Hospital Encounter,2012-04-26,EpicEncounterCsnId|DiagnosisId:ProfessionalCharge,NaT,Billing Diagnosis,No,No,Other dyspnea and respiratory abnormality,786.09,Other dyspnea and respiratory abnormality,R06.09,Other forms of dyspnea,*Not Applicable
4,SA90200020406974,NaT,NaT,Hospital Encounter,2011-12-24,EpicEncounterCsnId|DiagnosisId:ProfessionalCharge,NaT,Billing Diagnosis,No,No,Thoracic or lumbosacral neuritis or radiculiti...,724.4,Thoracic or lumbosacral neuritis or radiculiti...,IMO0002,NaN,*Not Applicable
5,SA90200020406974,NaT,NaT,Hospital Encounter,2012-03-11,EpicEncounterCsnId|DiagnosisId:EncounterDiagnosis,NaT,Encounter Diagnosis,No,No,Lumbosacral or thoracic radiculopathy,724.4,Thoracic or lumbosacral neuritis or radiculiti...,M54.10,"Radiculopathy, site unspecified",*Not Applicable
6,SA90200020406974,NaT,NaT,Office Visit,2012-05-07,EpicEncounterCsnId|DiagnosisId:EncounterDiagnosis,NaT,Encounter Diagnosis,No,No,Lumbar degenerative disc disease,722.52,Degeneration of lumbar or lumbosacral interver...,M51.36,"Other intervertebral disc degeneration, lumbar...",*Not Applicable
7,SA90200020406974,NaT,NaT,Office Visit,2012-10-15,EpicEncounterCsnId|DiagnosisId:ProfessionalCharge,NaT,Billing Diagnosis,No,No,"Open dislocation, lumbar vertebra",839.3,"Open dislocation, lumbar vertebra(839.30)",S33.101A,"Dislocation of unspecified lumbar vertebra, in...",*Not Applicable
8,SA90200020457330,NaT,NaT,Appointment,2012-01-23,EpicEncounterCsnId|DiagnosisId:EncounterDiagnosis,NaT,Encounter Diagnosis,No,No,"Hypertension, renal disease",585.9,"Chronic kidney disease, unspecified(585.9)",I12.9,Hypertensive chronic kidney disease with stage...,*Not Applicable
9,SA90200020457330,NaT,NaT,Appointment,2012-01-23,EpicEncounterCsnId|DiagnosisId:EncounterDiagnosis,NaT,Encounter Diagnosis,No,No,Immunosuppressive management encounter followi...,V58.69,Long-term (current) use of other medications,Z94.0,Kidney transplant status,*Not Applicable


In [6]:
offset = pd.read_csv('sameh_dateoffset.csv', sep='\t')

In [9]:
dx.shape

(130252, 16)

In [ ]:
dx_offset = dx.merge(offset, on = 'SAID')

#### Time shifted dates correctly

In [17]:
dx_offset._EncounterDischargeDate = dx_offset.apply(lambda x: x._EncounterDischargeDate + timedelta(days = x.date_offset), axis = 1)

dx_offset.Encounter_Hospital_Admission_Dat = dx_offset.apply(lambda x: x.Encounter_Hospital_Admission_Dat + timedelta(days = x.date_offset), axis = 1)

dx_offset.Diagnosis_Start_date = dx_offset.apply(lambda x: x.Diagnosis_Start_date + timedelta(days = x.date_offset), axis = 1)

dx_offset.Diagnosis_End_Date = dx_offset.apply(lambda x: x.Diagnosis_End_Date + timedelta(days = x.date_offset), axis = 1)

In [27]:
len(dx_offset[dx_offset.Diagnosis_End_Date.isnull()])

87568

## Diagnosis at Admission

### Take only dx present on dx

In [166]:
dx_offset_admission = dx_offset[dx_offset.Diagnosis_Present_On_Admission == 'Yes']

In [118]:
def remove_first_if_letter(x):
    if x[0].isalpha() and x != 'nan':
        return x[1:]
    else:
        return x

In [116]:
#dx_offset_admission['ICD9_THREE'] = dx_offset_admission.ICD9_Code.astype(str).apply(remove_first_if_letter)

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [137]:
dx_offset_admission['ICD9_THREE'] = dx_offset_admission.ICD9_Code.apply(lambda x: str(x).split(".")[0])

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [138]:
dx_offset_admission = dx_offset_admission[dx_offset_admission.ICD9_THREE != 'nan']

In [139]:
len(dx_offset_admission.ICD9_THREE.value_counts())

488

### Create mapping of ICD9 groupings to dx name

In [ ]:
import json
codes = json.load(open('icd9/codes.json', 'rb'))

In [195]:
def ICD9_group(level = 1):
    icd9_codes = []
    for seg in codes: 
        sample_code = {}
        if len(seg) > level:
            sample_code['descr'] = seg[level].get('descr')
            list_names = [x.values()[1] for x in icd9_codes if icd9_codes != []]
            #print list_names
            if sample_code['descr'] not in list_names:
                if seg[level]['code'] != None:
                    if '-' in seg[level]['code']:
                        sample_code['min'] = seg[level]['code'].split('-')[0]
                        sample_code['max'] = seg[level]['code'].split('-')[1]
                    else:
                        sample_code['min'] = seg[level]['code']
                        sample_code['max'] = seg[level]['code']
                    icd9_codes.append(sample_code)
    return icd9_codes

In [196]:
icd9_codes_0 = ICD9_group(0)
icd9_codes_2 = ICD9_group(2)

In [209]:
icd9_codes_0

[{'descr': u'INFECTIOUS AND PARASITIC DISEASES ',
  'max': u'139',
  'min': u'001'},
 {'descr': u'NEOPLASMS ', 'max': u'239', 'min': u'140'},
 {'descr': u'ENDOCRINE, NUTRITIONAL AND METABOLIC DISEASES, AND IMMUNITY DISORDERS ',
  'max': u'279',
  'min': u'240'},
 {'descr': u'MENTAL DISORDERS ', 'max': u'319', 'min': u'290'},
 {'descr': u'DISEASES OF THE NERVOUS SYSTEM AND SENSE ORGANS ',
  'max': u'389',
  'min': u'320'},
 {'descr': u'DISEASES OF THE CIRCULATORY SYSTEM ',
  'max': u'459',
  'min': u'390'},
 {'descr': u'DISEASES OF THE RESPIRATORY SYSTEM ',
  'max': u'519',
  'min': u'460'},
 {'descr': u'DISEASES OF THE DIGESTIVE SYSTEM ', 'max': u'579', 'min': u'520'},
 {'descr': u'DISEASES OF THE GENITOURINARY SYSTEM ',
  'max': u'629',
  'min': u'580'},
 {'descr': u'COMPLICATIONS OF PREGNANCY, CHILDBIRTH, AND THE PUERPERIUM ',
  'max': u'679',
  'min': u'630'},
 {'descr': u'DISEASES OF THE SKIN AND SUBCUTANEOUS TISSUE ',
  'max': u'709',
  'min': u'680'},
 {'descr': u'DISEASES OF THE

In [200]:
print len(icd9_codes_0)
print len(icd9_codes)
print len(icd9_codes_2)

14
152
1166


#### find right description by searching with ranges

In [144]:
def find_descr(x):
    #print x
    if "/" not in x:
        for code in icd9_codes:
            if x >= code['min'] and x <= code['max']:
                return code['descr']
    return None

In [151]:
dx_offset_admission[''] = dx_offset_admission.ICD9_THREE.apply(find_descr)

## Subset Data

In [163]:
# Only hospital encounters
dx_offset_admission = dx_offset_admission[dx_offset_admission['Encounter_Type'] == 'Hospital Encounter'].drop(['Encounter_Type'], inplace = True, axis = 1)

In [164]:
dx_offset_admission.shape

AttributeError: 'NoneType' object has no attribute 'shape'

In [157]:
len(dx_offset_admission['ICD9_THREE_descr'].value_counts()[dx_offset_admission['ICD9_THREE_descr'].value_counts() > 20])

58

## Final Diagnosis

In [ ]:
final_dx = dx_offset[dx_offset.Encounter_Type == 'Hospital Encounter']#.value_counts()

In [172]:
final_dx = final_dx[final_dx.Diagnosis_Event_Type == 'EpicHospitalAccountId|DiagnosisId:FinalDiagnosis']

In [181]:
final_dx = final_dx.drop(['ICD10_Code',
          'ICD10_Name',
           'Diagnosis_Event_Type',
            'Encounter_Type'],
          axis = 1)

In [182]:
final_dx['ICD9_THREE'] = final_dx.ICD9_Code.apply(lambda x: str(x).split(".")[0])

In [183]:
final_dx = final_dx[final_dx.ICD9_THREE != 'nan']

In [184]:
final_dx['dx_cat'] = final_dx.ICD9_THREE.apply(find_descr)

In [186]:
len(final_dx.dx_cat.value_counts())

102

In [ ]:
final_dx = final_dx.drop_duplicates(['dx_cat', 'SAID'])

In [300]:
#All dx groups
groups = list(final_dx.dx_cat.value_counts()[final_dx.dx_cat.value_counts() > 15].keys())
# groups = final_dx.dx_cat.unique()

In [310]:
remove_list = np.array(sorted(groups))[np.array([6, 18, 34, 39, 52, 54, 55, 56, 58, 59, 63])]
groups = list(set(groups) - set(remove_list))

In [311]:
pt_dx_ALL = []
for x in final_dx.SAID.unique():
    dx_list = list(final_dx[final_dx.SAID == x].dx_cat)
    #pt_dx['SA_ID'] = x
    line = list(np.in1d(groups, dx_list).astype(int)) + [x]
    pt_dx_ALL.append(line)

In [312]:
columns = list(groups) + ['SA_ID']

In [313]:
final_dx_df =  pd.DataFrame(data = pt_dx_ALL, columns = columns)

In [316]:
final_dx_df.shape

(444, 54)

In [317]:
final_dx_df.to_json('JSONs/final_dxs_recorded.json')

## Merge Dx DF to final DF

In [318]:
final_dx_df = pd.read_json('JSONs/final_dxs_recorded.json')

In [356]:
final_with_meds = pd.read_json('JSONs/final_HR_segments_with_meds_FULL_6_hrs.json')

In [357]:
final_with_meds_and_dx = final_with_meds.merge(final_dx_df, on = 'SA_ID')

#### get cols so sample_ID is first and pdf is last

In [358]:
cols = list(final_with_meds_and_dx)
cols.insert(0, cols.pop(cols.index('sample_ID')))
cols.append(cols.pop(cols.index('pdf')))
cols

[u'sample_ID',
 u'ABSORBABLE SULFONAMIDE ANTIBACTERIAL AGENTS',
 u'ADRENERGIC AGENTS,CATECHOLAMINES',
 u'ADRENERGIC VASOPRESSOR AGENTS',
 u'ALPHA/BETA-ADRENERGIC BLOCKING AGENTS',
 u'AMINOGLYCOSIDES',
 u'ANAEROBIC ANTIPROTOZOAL-ANTIBACTERIAL AGENTS',
 u'ANALGESIC/ANTIPYRETICS, SALICYLATES',
 u'ANALGESIC/ANTIPYRETICS,NON-SALICYLATE',
 u'ANALGESICS NARCOTIC, ANESTHETIC ADJUNCT AGENTS',
 u'ANALGESICS, NARCOTICS',
 u'ANDROGENIC AGENTS',
 u'ANTACIDS',
 u'ANTI-ANXIETY DRUGS',
 u'ANTIARRHYTHMICS',
 u'ANTICHOLINERGICS,QUATERNARY AMMONIUM',
 u'ANTICOAGULANTS,COUMARIN TYPE',
 u'ANTICONVULSANTS',
 u'ANTIDIARRHEAL MICROORGANISMS AGENTS',
 u'ANTIEMETIC/ANTIVERTIGO AGENTS',
 u'ANTIFLATULENTS',
 u'ANTIFUNGAL AGENTS',
 u'ANTIFUNGAL ANTIBIOTICS',
 u'ANTIHISTAMINES - 1ST GENERATION',
 u'ANTIHYPERLIPIDEMIC - HMG COA REDUCTASE INHIBITORS',
 u'ANTIHYPERTENSIVES, ACE INHIBITORS',
 u'ANTIHYPERTENSIVES, ANGIOTENSIN RECEPTOR ANTAGONIST',
 u'ANTIHYPERTENSIVES, SYMPATHOLYTIC',
 u'ANTIHYPERTENSIVES, VASODILATORS'

In [359]:
final_with_meds_and_dx = final_with_meds_and_dx.ix[:, cols]

In [360]:
final_with_meds_and_dx.to_json('JSONs/final_HR_segments_with_meds_and_dxs_FULL_6_hrs.json')

In [361]:
final_with_meds_and_dx = pd.read_json('JSONs/final_HR_segments_with_meds_and_dxs_FULL_6_hrs.json')


In [363]:
final_with_meds_and_dx.columns

Index([u'ABSORBABLE SULFONAMIDE ANTIBACTERIAL AGENTS',
       u'ADRENERGIC AGENTS,CATECHOLAMINES', u'ADRENERGIC VASOPRESSOR AGENTS',
       u'ALPHA/BETA-ADRENERGIC BLOCKING AGENTS', u'AMINOGLYCOSIDES',
       u'ANAEROBIC ANTIPROTOZOAL-ANTIBACTERIAL AGENTS',
       u'ANALGESIC/ANTIPYRETICS, SALICYLATES',
       u'ANALGESIC/ANTIPYRETICS,NON-SALICYLATE',
       u'ANALGESICS NARCOTIC, ANESTHETIC ADJUNCT AGENTS',
       u'ANALGESICS, NARCOTICS',
       ...
       u'VITAMIN D PREPARATIONS', u'VITAMIN K PREPARATIONS',
       u'ZINC REPLACEMENT', u'date', u'end_time', u'hist', u'length_of_block',
       u'pdf', u'sample_ID', u'start_time'],
      dtype='object', length=187)